In [1]:
import pandas as pd
from IPython.display import HTML, display

def display_html_table(df, max_height=300):
    """
    Retourne un bloc HTML avec une barre de défilement verticale (max_height).
    """
    html_str = f"""
    <div style="width:100%; max-height:{max_height}px; overflow:auto; border:1px solid #666; margin-bottom:10px;">
        {df.to_html(index=False, justify='left')}
    </div>
    """
    return HTML(html_str)


def analyze_trades(csv_file_path):
    # 1) Lecture du CSV
    df = pd.read_csv(csv_file_path)
    
    # Vérifie que les colonnes existent (ajuste selon tes noms de colonnes)
    # On part sur: 'instrument_name', 'trade_price', 'trade_amount', 'trade_fee', 'realized_pnl'
    required_cols = ['instrument_name','trade_price','trade_amount','trade_fee','realized_pnl']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"La colonne '{col}' est absente du CSV !")

    # 2) Ajout d'une colonne "notional" = trade_price * trade_amount
    df['notional'] = df['trade_price'] * df['trade_amount']

    # 3) Groupby par instrument_name : calcul des stats
    grouped = df.groupby('instrument_name').agg(
        total_volume   = pd.NamedAgg(column='notional',       aggfunc='sum'),
        total_pnl      = pd.NamedAgg(column='realized_pnl',   aggfunc='sum'),
        total_fees     = pd.NamedAgg(column='trade_fee',      aggfunc='sum'),
        winning_trades = pd.NamedAgg(column='realized_pnl',   aggfunc=lambda s: (s > 0).sum()),
        losing_trades  = pd.NamedAgg(column='realized_pnl',   aggfunc=lambda s: (s < 0).sum()),
        biggest_gain   = pd.NamedAgg(column='realized_pnl',   aggfunc='max'),
        biggest_loss   = pd.NamedAgg(column='realized_pnl',   aggfunc='min'),
    ).reset_index()

    # 4) Statistiques globales
    global_stats = {
        'nb_total_trades': len(df),
        'global_volume': df['notional'].sum(),
        'global_pnl': df['realized_pnl'].sum(),
        'global_fees': df['trade_fee'].sum()
    }
    global_stats_df = pd.DataFrame([global_stats])

    # 5) Top 5 plus gros gains / plus grosses pertes (au niveau des trades individuels)
    top_gains = df.nlargest(5, 'realized_pnl')
    top_losses = df.nsmallest(5, 'realized_pnl')

    # ---- AFFICHAGE HTML ----
    # A) Statistiques globales
    display(HTML("<h2>Statistiques globales</h2>"))
    display(display_html_table(global_stats_df, max_height=150))
    
    # B) Résumé par instrument
    display(HTML("<h2>Récap par instrument</h2>"))
    display(display_html_table(grouped))
    
    # C) Top 5 plus gros gains
    display(HTML("<h2>Top 5 plus gros gains (trades individuels)</h2>"))
    display(display_html_table(top_gains[['trade_id','instrument_name','realized_pnl','trade_price','trade_amount']], max_height=200))
    
    # D) Top 5 plus grosses pertes
    display(HTML("<h2>Top 5 plus grosses pertes (trades individuels)</h2>"))
    display(display_html_table(top_losses[['trade_id','instrument_name','realized_pnl','trade_price','trade_amount']], max_height=200))


# ---------------------------------------------------------------------------
# Exemple d'utilisation (Notebook/Jupyter) :
if __name__ == "__main__":
    csv_file = r"C:\Users\nicol\Desktop\Algotrading_2025\DERIVE.XYZ\extraction_trades2\PERPS-ALT_1741737600000_1744156800000\trades_76Fb9.csv"
    analyze_trades(csv_file)


nb_total_trades,global_volume,global_pnl,global_fees
3398,7.395438e+06,7562.206903,1337.904184


instrument_name,total_volume,total_pnl,total_fees,winning_trades,losing_trades,biggest_gain,biggest_loss
AAVE-PERP,3.578387e+05,1108.599625,51.370833,41,52,314.007081,-147.457084
ARB-PERP,9.544196e+04,1340.867342,14.855634,21,7,206.089507,-197.240276
AVAX-PERP,1.839627e+05,-26.083140,30.935663,17,37,362.220936,-457.916586
BNB-PERP,9.242022e+05,4739.758098,169.784812,202,69,193.473909,-76.639391
ENA-PERP,2.708471e+05,2127.440668,36.495011,70,65,476.714044,-383.369547
LINK-PERP,2.839632e+05,-5703.194176,50.227836,11,83,176.319300,-586.578641
NEAR-PERP,7.350874e+04,442.114576,9.569894,9,6,265.551910,-303.290088
OP-PERP,4.019350e+05,-1444.462718,74.035595,28,53,122.941988,-136.718674
PEPE-PERP,5.294870e+04,-3942.772504,3.987885,0,15,0.000000,-632.297782
SUI-PERP,3.203798e+05,3020.366650,59.902397,46,21,211.288733,-313.049435


trade_id,instrument_name,realized_pnl,trade_price,trade_amount
1fac9f0c-132c-4d18-886a-1cdbbad3bc24,ENA-PERP,476.714044,0.42964,11842.40
a60b206e-93e8-4371-92c3-fbac8354728b,ENA-PERP,417.096732,0.41494,12262.00
09458d79-b460-4862-b6c9-43839eb3e399,AVAX-PERP,362.220936,22.77590,126.81
28007f00-89e8-443f-8cb9-8a17fd4298d3,AVAX-PERP,362.192371,22.77590,126.80
960cf8dc-8eab-4e15-9091-723d3322916d,ENA-PERP,345.653916,0.41549,10000.00


trade_id,instrument_name,realized_pnl,trade_price,trade_amount
0657564e-fa4c-4777-941a-861941ed5b54,UNI-PERP,-950.651535,5.918900,8.596100e+02
db446007-4441-4b10-af4e-c1152b03e341,PEPE-PERP,-632.297782,0.000008,6.744400e+08
a9f8f3d9-df2c-43fe-8351-ea7bbab24300,UNI-PERP,-592.547303,5.831400,4.960300e+02
29511f57-586f-4fba-abf1-262c3626f3f0,LINK-PERP,-586.578641,13.304300,3.824320e+02
73a61568-d0e6-42f2-8943-c332f3d32939,UNI-PERP,-507.838913,5.975300,4.833300e+02


In [ ]:
# -*- coding: utf-8 -*-
"""
Wash‑Trading Detector — Console‑Only (2025‑06)
==============================================

• Parcourt tous les sous‑dossiers de `EXTRACTION_ROOT` dont le nom suit
  `PERPS-(ALT|MAJ)_<epochStart>_<epochEnd>` (ignore `OPTIONS-*`).
• Charge les adresses Market‑Makers (MM) depuis le JSON fourni.
• Construit un graphe maker→taker, détecte :
    – cycles courts (≤3 nœuds) ;
    – échanges bidirectionnels rapprochés (< 1 h) ;
    – adresses à ratio bidirectionnel > 0,5.
• Affiche :
    – pour chaque programme → 3 tableaux (cycles, bidirectionnels, ratios) +
      un tableau récapitulatif par adresse suspecte + mini‑rapport par adresse ;
    – enfin une synthèse globale (tous programmes).

Aucun fichier n’est écrit. Tout est dans la console → idéal pour CI/log.

Exécution :
    python wash_trading_console.py
"""

from __future__ import annotations

import os
import json
from datetime import datetime
from collections import defaultdict, Counter
from typing import Dict, List, Tuple

import pandas as pd
import networkx as nx
import requests
from tqdm import tqdm

###############################################################################
# CONFIG UTILISATEUR                                                          #
###############################################################################
EXTRACTION_ROOT = r"C:/Users/nicol/Desktop/Algotrading_2025/DERIVE.XYZ/extraction_trades"
MM_JSON_PATH    = r"C:/Users/nicol/Desktop/Algotrading_2025/DERIVE.XYZ/market_maker_data/all_mm_addresses_by_program.json"
API_URL         = "https://api.lyra.finance/public/get_trade_history"

PAIRS_BY_PROGRAM: Dict[str, List[str]] = {
    "PERPS-ALT": [
        "NEAR-PERP", "TAO-PERP", "SUI-PERP", "LINK-PERP", "XRP-PERP", "ARB-PERP",
        "WLD-PERP", "DEGEN-PERP", "SEI-PERP", "TIA-PERP", "AAVE-PERP", "OP-PERP",
        "BNB-PERP", "UNI-PERP", "AVAX-PERP", "EIGEN-PERP", "ENA-PERP", "PEPE-PERP",
        "BITCOIN-PERP", "WIF-PERP", "TRUMP-PERP",
    ],
    "PERPS-MAJ": ["ETH-PERP", "SOL-PERP", "BTC-PERP"],
}

TIME_THRESHOLD = 3600  # secondes (1 h) pour la fenêtre bidirectionnelle
TOP_N_PATTERN  = 10    # lignes à afficher pour chaque tableau
TOP_N_ADDRESS  = 10    # adresses à détailler
###############################################################################
# OUTILS D’AFFICHAGE                                                          #
###############################################################################

def print_df(df: pd.DataFrame, title: str):
    """Imprime un DataFrame joliment formaté ou ‘aucun résultat’."""
    print(f"\n{title} (n={len(df)})" + ("" if not df.empty else " — aucun résultat"))
    if not df.empty:
        with pd.option_context("display.max_columns", None, "display.width", 0):
            print(df.to_string(index=False))

###############################################################################
# 1. DÉTECTION DES SCHÉMAS                                                   #
###############################################################################

def detect_patterns(G: nx.DiGraph, window: int = TIME_THRESHOLD):
    patterns: List[dict] = []

    # ---- cycles courts --------------------------------------------------
    raw_cycles = [c for c in nx.simple_cycles(G) if len(c) <= 3]
    uniq_cycles = {tuple(c[c.index(min(c)):]+c[:c.index(min(c))]) for c in raw_cycles}
    for cyc in uniq_cycles:
        vol = sum(G[cyc[i]][cyc[(i+1)%len(cyc)]].get("volume", 0) for i in range(len(cyc)))
        trades = [t for i in range(len(cyc)) for t in G[cyc[i]][cyc[(i+1)%len(cyc)]].get("trades", [])]
        patterns.append({"type": "cycle", "addresses": cyc, "volume": vol, "trades": trades})

    # ---- bidirectionnels rapprochés -------------------------------------
    visited = set()
    for u, v in G.edges():
        pair = tuple(sorted((u, v)))
        if pair in visited or not G.has_edge(v, u):
            continue
        visited.add(pair)
        close_pairs = []
        for t1 in G[u][v]["trades"]:
            for t2 in G[v][u]["trades"]:
                if t1["timestamp"] and t2["timestamp"]:
                    delta = abs((t1["timestamp"] - t2["timestamp"]).total_seconds())
                    if delta < window:
                        close_pairs.append({"fwd": t1, "bwd": t2, "dt": delta})
        if close_pairs:
            tot_vol = G[u][v]["volume"] + G[v][u]["volume"]
            patterns.append({"type": "bidi", "addresses": pair, "volume": tot_vol, "pairs": close_pairs})

    # ---- ratios élevés ---------------------------------------------------
    for n in G.nodes():
        preds, succs = set(G.predecessors(n)), set(G.successors(n))
        all_neigh = preds | succs
        if all_neigh:
            ratio = len(preds & succs) / len(all_neigh)
            if ratio > 0.5 and len(all_neigh) > 1:
                patterns.append({"type": "ratio", "address": n, "ratio": ratio, "neighbors": list(preds & succs)})

    return sorted(patterns, key=lambda p: p.get("volume", 0), reverse=True)

###############################################################################
# 2. GRAPHE DE TRANSACTIONS                                                  #
###############################################################################

def build_graph(df_maker: pd.DataFrame, df_taker: pd.DataFrame, mm_set: set[str]):
    G = nx.DiGraph()
    edge_acc: Dict[Tuple[str, str], dict] = defaultdict(lambda: {"count": 0, "volume": 0.0, "trades": []})

    maker_map = {r.trade_id: r for r in df_maker.itertuples(index=False)}
    for r in df_maker.itertuples(index=False):
        G.add_node(r.wallet, trades=G.nodes.get(r.wallet, {}).get("trades", 0)+1, is_mm=r.wallet in mm_set)

    for r in df_taker.itertuples(index=False):
        if r.trade_id not in maker_map:
            continue
        m_wallet, t_wallet = maker_map[r.trade_id].wallet, r.wallet
        for w in (t_wallet,):
            G.add_node(w, trades=G.nodes.get(w, {}).get("trades", 0)+1, is_mm=w in mm_set)
        vol = getattr(r, "volume_usdt", 0.0)
        meta = edge_acc[(m_wallet, t_wallet)]
        meta["count"] += 1
        meta["volume"] += vol
        meta["trades"].append({"trade_id": r.trade_id, "timestamp": getattr(r, "datetime", None), "volume": vol})

    for (u, v), d in edge_acc.items():
        G.add_edge(u, v, **d)
    return G

###############################################################################
# 3. CHARGEMENT DES DONNÉES                                                  #
###############################################################################

def _prepare(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    if {"trade_price", "trade_amount"}.issubset(df.columns):
        df["volume_usdt"] = pd.to_numeric(df.trade_price, errors="coerce") * pd.to_numeric(df.trade_amount, errors="coerce")
    if "timestamp" in df.columns:
        df["datetime"] = pd.to_datetime(df.timestamp, errors="coerce", unit="ms" if df.timestamp.dtype != object else None)
    return df


def fetch_trades_api(pairs: List[str], ts_from: int, ts_to: int) -> pd.DataFrame:
    all_trades = []
    max_pages = 10  # limite de sécurité
    max_errors = 3

    for pair in tqdm(pairs, desc="API", leave=False):
        page = 1
        error_count = 0
        while page <= max_pages:
            payload = {
                "instrument_name": pair,
                "from_timestamp": ts_from,
                "to_timestamp": ts_to,
                "instrument_type": "perp",
                "page": page,
                "page_size": 1000,
            }
            try:
                r = requests.post(API_URL, json=payload, timeout=10)
                r.raise_for_status()
                res = r.json().get("result", {})
                trades = res.get("trades", []) if isinstance(res, dict) else res
                if not trades:
                    break
                all_trades.extend(trades)
                if len(trades) < 1000:
                    break
                page += 1
            except Exception as e:
                error_count += 1
                print(f"Erreur API sur {pair} (page {page}) : {str(e)}")
                if error_count >= max_errors:
                    print(f"Abandon de la paire {pair} après {error_count} erreurs.")
                    break
    return _prepare(pd.DataFrame(all_trades))



def load_data(csv_files: List[str], pairs: List[str], ts_from: int, ts_to: int):
    df_csv = pd.concat([pd.read_csv(p) for p in csv_files], ignore_index=True)
    trade_ids = df_csv[df_csv.liquidity_role.str.lower()=="maker"].trade_id.unique()

    df_api = fetch_trades_api(pairs, ts_from, ts_to)
    if df_api.empty:
        return pd.DataFrame(), pd.DataFrame()

    df_api = df_api[df_api.trade_id.isin(trade_ids)]
    return (
        _prepare(df_api[df_api.liquidity_role.str.lower()=="maker"]),
        _prepare(df_api[df_api.liquidity_role.str.lower()=="taker"]),
    )

###############################################################################
# 4. ANALYSE D’UN PROGRAMME                                                 #
###############################################################################

def analyse_program(folder: str, mm_map: Dict[str, List[str]]):
    parts = folder.split("_")
    if len(parts) != 3:
        return None  # format inattendu
    program_name, start_ts_str, end_ts_str = parts[0], parts[1], parts[2]
    if program_name not in PAIRS_BY_PROGRAM:
        return None  # ignore OPTIONS et inconnus

    start_ts = int(start_ts_str)
    key_mm = f"{program_name}_{start_ts}"
    mm_list = mm_map.get(key_mm, [])

    csv_folder = os.path.join(EXTRACTION_ROOT, folder)
    if not os.path.isdir(csv_folder):
        return None
    csv_files = [os.path.join(csv_folder, f) for f in os.listdir(csv_folder) if f.endswith(".csv")]
    if not csv_files:
        return None

    # Header programme
    print("\n" + "="*80)
    date_str = datetime.fromtimestamp(start_ts/1000).strftime("%Y-%m-%d")
    print(f"PROGRAMME {program_name} | epoch {date_str} | CSV {len(csv_files)} | MM {len(mm_list)}")
    print("="*80)

    df_maker, df_taker = load_data(csv_files, PAIRS_BY_PROGRAM[program_name], start_ts, int(end_ts_str))
    if df_maker.empty or df_taker.empty:
        print("  → aucune donnée exploitable (API ou CSV)")
        return None

    G = build_graph(df_maker, df_taker, set(mm_list))
    patterns = detect_patterns(G)

    # Tableaux patterns ---------------------------------------------------
    cycles = [p for p in patterns if p["type"] == "cycle"]
    bidis  = [p for p in patterns if p["type"] == "bidi"]
    ratios = [p for p in patterns if p["type"] == "ratio"]

    print_df(pd.DataFrame([
        {"ID": i+1, "Chemin": " -> ".join(p["addresses"]) + " -> " + p["addresses"][0], "Volume": p["volume"], "Trades": len(p["trades"])}
        for i, p in enumerate(cycles)
    ]).head(TOP_N_PATTERN), "CYCLES COURTS")

    print_df(pd.DataFrame([
        {"ID": i+1, "Adresses": " <-> ".join(p["addresses"]), "Volume": p["volume"], "Paires": len(p["pairs"])}
        for i, p in enumerate(bidis)
    ]).head(TOP_N_PATTERN), "BIDIRECTIONNELS <1h")

    print_df(pd.DataFrame([
        {"ID": i+1, "Adresse": p["address"], "Ratio": f"{p['ratio']:.2f}", "Voisins": len(p["neighbors"])}
        for i, p in enumerate(ratios)
    ]).head(TOP_N_PATTERN), "ADRESSES RATIO>0.5")

    # Récap par adresse suspecte -----------------------------------------
    addr_stats: Dict[str, dict] = defaultdict(lambda: {"patterns": 0, "volume": 0.0, "cycles": 0, "bidis": 0, "ratios": 0})
    for p in patterns:
        if p["type"] == "cycle":
            for a in p["addresses"]:
                s = addr_stats[a]; s["patterns"] += 1; s["volume"] += p["volume"]; s["cycles"] += 1
        elif p["type"] == "bidi":
            for a in p["addresses"]:
                s = addr_stats[a]; s["patterns"] += 1; s["volume"] += p["volume"]; s["bidis"] += 1
        else:
            a = p["address"]
            s = addr_stats[a]; s["patterns"] += 1; s["ratios"] += 1

    addr_df = pd.DataFrame([
        {"Adresse": a, **stats} for a, stats in addr_stats.items()
    ]).sort_values(["patterns", "volume"], ascending=False)

    print_df(addr_df.head(TOP_N_ADDRESS), "ADRESSES LES PLUS SUSPECTES")

    # Mini‑rapport par adresse (affichage détaillé)
    for _, row in addr_df.head(TOP_N_ADDRESS).iterrows():
        print("\n" + "-"*60)
        print(f"ADRESSE SUSPECTE : {row['Adresse']}")
        print("-"*60)
        print(f"  • Patterns totaux     : {row['patterns']}")
        print(f"  • Volume suspect      : ${row['volume']:,.2f}")
        print(f"    – Cycles            : {row['cycles']}")
        print(f"    – Bidirectionnels   : {row['bidis']}")
        print(f"    – Ratios élevés     : {row['ratios']}")

    return {
        "program_name": program_name,
        "date": date_str,
        "patterns_count": len(patterns),
        "cycles": len(cycles),
        "bidis": len(bidis),
        "ratios": len(ratios),
        "total_volume": sum(p.get("volume", 0) for p in patterns),
    }

###############################################################################
# 5. BOUCLE GLOBALE                                                          #
###############################################################################

def run_all_programs():
    # Vérifier chemins
    if not os.path.isdir(EXTRACTION_ROOT):
        raise FileNotFoundError(f"EXTRACTION_ROOT introuvable: {EXTRACTION_ROOT}")
    if not os.path.isfile(MM_JSON_PATH):
        raise FileNotFoundError(f"MM_JSON_PATH introuvable: {MM_JSON_PATH}")

    with open(MM_JSON_PATH, "r", encoding="utf-8") as f:
        mm_map = json.load(f)

    folders = [d for d in os.listdir(EXTRACTION_ROOT) if os.path.isdir(os.path.join(EXTRACTION_ROOT, d))]
    folders.sort()

    global_results = []
    for folder in folders:
        res = analyse_program(folder, mm_map)
        if res:
            global_results.append(res)

    # ------------------------------ Synthèse globale ---------------------------
    if global_results:
        total_prog = len(global_results)
        tot_patterns = sum(r["patterns_count"] for r in global_results)
        tot_cycles   = sum(r["cycles"] for r in global_results)
        tot_bidis    = sum(r["bidis"] for r in global_results)
        tot_ratios   = sum(r["ratios"] for r in global_results)
        tot_volume   = sum(r["total_volume"] for r in global_results)

        top_prog = max(global_results, key=lambda r: r["patterns_count"])["program_name"] if global_results else None

        print("\n" + "="*80)
        print("SYNTHÈSE GLOBALE")
        print("="*80)
        print(f"Programmes analysés          : {total_prog}")
        print(f"Patterns suspects (total)    : {tot_patterns}")
        print(f"Volume total suspect         : ${tot_volume:,.2f}")
        print("Répartition des patterns     :")
        print(f"  – Cycles                  : {tot_cycles}")
        print(f"  – Bidirectionnels <1h     : {tot_bidis}")
        print(f"  – Ratios élevés           : {tot_ratios}")
        if top_prog:
            print(f"\nProgramme le plus problématique : {top_prog}")

        # Détail top 3
        df_glob = pd.DataFrame(global_results)
        df_top = df_glob.sort_values("patterns_count", ascending=False).head(3)
        if not df_top.empty:
            print("\nTop 3 programmes par patterns suspects :")
            for i, row in enumerate(df_top.itertuples(index=False), 1):
                print(f"{i}. {row.program_name} ({row.date}) – {row.patterns_count} patterns, volume ${row.total_volume:,.2f}")

        # Recommandations de base
        print("\nRECOMMANDATIONS :")
        recos = [
            "Inspecter manuellement les programmes avec >100 patterns et/ou volume élevé",
            "Identifier si les adresses apparaissent dans plusieurs programmes",
            "Surveiller en continu les cycles courts récurrents",
            "Mettre en place seuils dynamiques sur ratio bidirectionnel",
            "Réviser les conditions d'éligibilité des market-makers",
        ]
        for i, txt in enumerate(recos, 1):
            print(f"{i}. {txt}")

        print("\nATTENTION : ces heuristiques signalent des anomalies, pas des preuves. Une revue humaine reste indispensable.")
    else:
        print("\nAucun programme exploitable n'a été trouvé ou aucune donnée retour API.")

###############################################################################
# 6. MAIN                                                                     #
###############################################################################

if __name__ == "__main__":
    print("ANALYSE DE WASH TRADING — VERSION CONSOLE SEULE")
    run_all_programs()

